In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown, update_display
from openai import OpenAI


In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

model = "gpt-4o-mini"
openai = OpenAI()

In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

In [5]:
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON. If I provide you with a list of links such as the one below:\n"
link_system_prompt += """
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
mailto:hello@mygroovydomain.com
https://www.linkedin.com/in/eddonner/
https://twitter.com/edwarddonner
https://www.facebook.com/edward.donner.52
"""

link_system_prompt += "You should respond with a JSON object that looks like this:\n"

link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"},
        {"type": "posts page", "url": "https://edwarddonner.com/posts/"},
        {"type": "find me on linkedin", "url": "https://www.linkedin.com/in/eddonner/"}
        ]
}
"""

link_system_prompt += "And another example:\n"
link_system_prompt += "If I provide you with a list of links such as the one below:\n"

link_system_prompt += """
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://www.anthropic.com/news/visible-extended-thinking',
 'https://www.anthropic.com/transparency',
 'https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/anthropic-achieves-iso-42001-certification-for-responsible-ai',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/engineering',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/company',
 'https://www.anthropic.com/careers',
 'https://www.anthropic.com/events',
 'https://www.anthropic.com/engineering/building-effective-agents',
 'https://www.anthropic.com/news',
 '#',
 'https://claude.ai/login',
 'https://claude.ai/download',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://www.anthropic.com/news/visible-extended-thinking',
 'https://www.anthropic.com/transparency',
 'https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/anthropic-achieves-iso-42001-certification-for-responsible-ai',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/engineering',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/company',
 'https://www.anthropic.com/careers',
 'https://www.anthropic.com/events',
 'https://www.anthropic.com/engineering/building-effective-agents',
 'https://www.anthropic.com/news',
 '#',
 'https://claude.ai/login',
 'https://claude.ai/download',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 'https://www.anthropic.com/api',
 'https://anthropic.com/news/claude-4',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/core-views-on-ai-safety',
 'https://www.anthropic.com/rsp-updates',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/research/project-vend-1',
 'https://www.anthropic.com/research/agentic-misalignment',
 'https://www.anthropic.com/news/claude-4',
 'https://www.anthropic.com/research/tracing-thoughts-language-model',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/news/model-context-protocol',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/contact-sales',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/claude-code',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://claude.ai/download',
 'https://www.anthropic.com/pricing',
 'http://claude.ai/login',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/amazon-bedrock',
 'https://www.anthropic.com/google-cloud-vertex-ai',
 'https://www.anthropic.com/pricing#anthropic-api'
 """

link_system_prompt += "You should respond with a JSON object that looks like this:\n"

link_system_prompt += """
{
    "links": [
        {"type": "our team", "url": "https://www.anthropic.com/team"},
        {"type": "events page", "url": "https://www.anthropic.com/events"},
        {"type": "about company page", "url": "https://www.anthropic.com/company"},
        {"type": "careers page", "url": "https://www.anthropic.com/careers"},
        {"type": "contact sales page", "url": "https://www.anthropic.com/contact-sales"},
        {"type": "claude 4", "url": "https://www.anthropic.com/news/claude-4"}
    ]
}
"""

In [7]:
print(link_system_prompt) 

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON. If I provide you with a list of links such as the one below:

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-ex

In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=model,
        messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [11]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'posts page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'find me on linkedin',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [12]:
get_links("https://anthropic.com")

{'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'},
  {'type': 'our team', 'url': 'https://www.anthropic.com/team'},
  {'type': 'enterprise solutions',
   'url': 'https://www.anthropic.com/enterprise'},
  {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'},
  {'type': 'API page', 'url': 'https://www.anthropic.com/api'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'company page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'learn more page', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'},
  {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'},
  {'type': 'transparency page',
   'url': 'https://www.anthropic.com/transparency'},
  {'type': 'solutions page',
   'url': 'h

In [ ]:
anthropic = Website("https://anthropic.com")
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonne

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links", links)
    for link in links['links']:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()
    return result

print(get_all_details("https://anthropic.com"))

Found links {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Education
Financial services
Case studies
Hear from our customers
Research
Research

In [26]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here iare the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt += user_prompt[:20_000]
    return user_prompt

In [29]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}]}


"You are looking at a company called: Anthropic\nHere iare the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nEducation\nFinancial services\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nIS

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [31]:
create_brochure("Anthropic", "https://anthropic.com")

Found links {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}


# Anthropic Company Brochure

---

## Welcome to Anthropic

At Anthropic, we are dedicated to building AI systems that prioritize safety, reliability, and human well-being. Our flagship model, **Claude**, showcases our commitment to innovative AI technology while ensuring it serves humanity’s long-term interests.

### Core Mission

We believe that AI can have profound impacts on the world, and our mission focuses on creating systems people can trust. By emphasizing both opportunities and risks associated with AI, we aim to set industry standards for responsible AI development.

---

## Our Products

- **Claude Models**: 
  - **Claude Opus 4**: Our most advanced model designed for complex reasoning and coding tasks.
  - **Claude Sonnet 4**: Offers robust capabilities suitable for a variety of applications.
  - **Claude Haiku 3.5**: A versatile model catering to general needs.

- **Anthropic Academy**: Offers resources and courses to help our users leverage Claude effectively for their projects.

- **Enterprise Solutions**: Claude serves various sectors, including finance, education, and customer support, providing tailored tools for collaboration and efficiency.

---

## Interdisciplinary Team

At Anthropic, our team consists of experts in various fields, including:
- Research & Engineering
- Policy and Ethics
- Business Leadership
- Technical Operations

We function as a cohesive unit, treating AI safety as a systematic science and continuously sharing insights to promote a safer AI ecosystem.

---

## Company Values

1. **Act for the Global Good**: We aim to maximize positive outcomes for humanity through bold technological advancements.
2. **Hold Light and Shade**: We acknowledge the potential risks of AI while striving to harness its benefits responsibly.
3. **Be Good to Our Users**: We foster kindness and generosity in our interactions with all stakeholders.
4. **Ignite a Race to the Top on Safety**: We strive to set industry standards for AI safety and security.
5. **Keep it Simple**: We focus on empirical solutions that have a substantial impact.
6. **Be Helpful, Honest, and Harmless**: Open communication and high trust characterize our work culture.
7. **Put the Mission First**: Our mission guides every action we take toward our collective goal.

---

## Careers at Anthropic

Join us in Building the Future of Safe AI

We are continually expanding our team and welcome individuals from diverse backgrounds. Our values create a supportive work environment where creativity and collaboration thrive.

### Benefits:
- Comprehensive health and wellness programs
- Flexible paid time off policies
- Competitive salaries and equity packages
- Career development resources through Anthropic Academy

**Explore Open Roles**: Interested in joining us? Learn about our current job openings [here](#).

---

## Customer Success Stories

We are proud to partner with numerous enterprises and startups that utilize Claude to enhance their operations. Some highlights include:
- **Gradient Labs**: Transformed customer support in financial services.
- **ClassDojo**: Improved efficiency for educators.
- **Zapier**: Fostered an AI-first remote culture.

For more success stories and case studies, visit our [customer page](#).

---

## Join Us and Shape the Future

Become part of a team that is not only pioneering in AI technology but also committed to its responsible development. Let’s make a positive impact on the world together.

For more information, please visit our website at [Anthropic.com](#).

---

**Contact Us**: For inquiries or collaboration opportunities, reach out at support@anthropic.com.

--- 

*Thank you for exploring Anthropic. Together, we can make AI safer and more beneficial for everyone.*

In [32]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )

    response = ""

    display_handle = display(Markdown(""),  display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'about page', 'url': 'https://www.anthropic.com/about'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}]}


# Anthropic Brochure

## About Anthropic
Anthropic is a leading artificial intelligence safety and research organization focused on developing reliable, interpretable, and steerable AI systems. Founded with the mission to advance AI technology while prioritizing safety and human benefit, Anthropic operates as a Public Benefit Corporation headquartered in San Francisco.

---

## Our Vision
At Anthropic, we believe in the transformative potential of AI and its ability to create significant benefits for society. We are committed to ensuring that AI development is responsible, transparent, and ethical. With products such as Claude, our flagship AI model, we aim to make AI systems that can power numerous applications while providing users with safety and dependability.

---

## Core Values
1. **Act for the Global Good**: Making bold decisions that maximize positive outcomes for humanity.
2. **Hold Light and Shade**: Recognizing the dual potential of AI to pose risks and offer unparalleled benefits.
3. **Be Good to Our Users**: Cultivating kindness and generosity in all interactions.
4. **Ignite a Race to the Top on Safety**: Leading the industry towards the highest standards of AI safety.
5. **Do the Simple Thing That Works**: Focusing on effective and practical solutions.
6. **Be Helpful, Honest, and Harmless**: Promoting open communication and building a high-trust environment.
7. **Put the Mission First**: Collaboratively working towards a shared purpose.

---

## Products
### Claude
Claude is Anthropic's AI language model, designed to assist users in various tasks, from coding and customer support to educational applications. The current versions include:
- **Claude Opus 4**
- **Claude Sonnet 4**
- **Claude Haiku 3.5**

### Applications
Claude is versatile across multiple sectors, working to enhance productivity in:
- Education
- Financial Services
- Customer Support
- Software Development

---

## Customers
Anthropic proudly serves a diverse clientele, including prominent enterprises and innovative startups. Our customers span industries such as healthcare, finance, and technology, leveraging Claude for superior operational efficiencies and enhancing user experiences. Case studies showcase how organizations have transformed their workflows, utilizing Claude for applications ranging from ticketing solutions to collaborative software development.

---

## Company Culture and Careers
### Join Our Team
We are dedicated to building a cohesive and interdisciplinary team. Our workforce includes professionals from various backgrounds including engineering, research, public policy, and business. We value diverse experiences and encourage candidates from all backgrounds to apply.

### Benefits
Anthropic offers a range of employee benefits designed to support the well-being and growth of our team members. Benefits include:
- Comprehensive health, dental, and vision insurance
- Paid parental leave and mental health support
- A flexible wellness stipend
- Home office and relocation support

---

## Commitment to AI Safety 
The foundation of our operations is built on a robust commitment to AI safety. We treat safety as a systematic science, where ongoing research feeds directly into product development, and insights gained are shared across the AI community.

---

## Get Involved
If you're interested in joining us on our mission to shape a positive future with AI, visit our careers page to explore open roles and become part of the Anthropic team!

---

### Follow Us
Stay updated on our latest developments, industry insights, and upcoming events by visiting our website at [Anthropic](https://www.anthropic.com) or follow us on various social media platforms! 

---

Join us in building safer AI for a better tomorrow!

In [34]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

## Overview
### Hugging Face – The AI Community Building the Future
Founded in 2016, Hugging Face is a leading AI company dedicated to democratizing artificial intelligence through innovative machine learning solutions. Our platform enables collaboration within the machine learning community, empowering users to build and share state-of-the-art models and datasets.

## Core Offerings
- **Models**: Browse over 1 million models, including trending AI applications and the latest innovations in natural language processing, computer vision, and more.
- **Datasets**: Access a rich repository of over 250,000 datasets for various ML tasks, facilitating easier collaboration and research.
- **Spaces**: Use our platform to create and deploy applications, ranging from simple demos to complex AI projects.

## Company Culture
### Mission-Driven Community
At Hugging Face, our mission is to foster a culture of shared knowledge and collaborative development. We believe in the power of open-source technology, empowering developers to leverage AI responsibly and effectively. Our community thrives on innovation and harnesses the collective intelligence and creativity of our users.

### Inclusivity & Growth
We embrace a diverse workforce and promote inclusivity in all aspects of our operations. We are committed to continuous learning and personal growth, both for our employees and community members.

## Customer Base
Hugging Face is trusted by over **50,000 organizations**, including major enterprises like:
- **Google**
- **Microsoft**
- **Amazon**
- **Meta**
These organizations leverage our robust AI tools for a variety of applications, solidifying our reputation as a leader in the AI community.

## Join Our Team
### Careers
We are always looking for talented individuals who are passionate about AI and eager to make a difference. We offer a vibrant work environment focused on collaboration, growth, and innovation. 

- **Open Positions**: Explore current career opportunities [here](https://huggingface.co/jobs).
- **Employee Benefits**: Enjoy flexible working arrangements, generous leave policies, and opportunities for professional development.

### Connect with Us
To learn more about what we do or to get involved:
- Visit our website: [huggingface.co](https://huggingface.co)
- Follow us on [LinkedIn](https://www.linkedin.com/company/huggingface)

## Conclusion
At Hugging Face, we are committed to advancing the future of artificial intelligence. By fostering a community-driven approach, we empower innovators and change-makers to collaborate and create impactful solutions that benefit all.

*Join us in shaping the future of AI.*

In [37]:
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonne